In [1]:
import os
import random
import copy
import env_vars
import csv
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from scipy import spatial

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/anon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<h2>Configurations Vector Space Modelling

<h3>Incidence Dictionary

Load Extracted CSV files

In [2]:
configs = None
with open(env_vars.PROJ_HOME + 'src/VSM/parser/output_copter.csv') as csv_file:
    configs = {}
    file_data = csv.reader(csv_file, delimiter=',')
    headers = next(file_data)
    for row in file_data:
        row_dict = dict(zip(headers, row))

        if row_dict['Description'] != '':
            configs[row[0]] = {
                'name': row_dict['Name'],
                'description': row_dict['Name'] + " " + row_dict['Description'],
                'range': [] if row_dict['Range'] == '' else  row_dict['Range'].split(' '),
                'increment': None if row_dict['Increment'] == '' else float(row_dict['Increment']), 
                'value': [] if row_dict['Value'] == '' else  [v for v in row_dict['Value'].split('|') if v != '']
            }
configs

{'SYSID_THISMAV': {'name': 'SYSID_THISMAV',
  'description': 'SYSID_THISMAV Allows setting an individual MAVLink system id for this vehicle to distinguish it from others on the same network',
  'range': ['1', '255'],
  'increment': None,
  'value': []},
 'SYSID_MYGCS': {'name': 'SYSID_MYGCS',
  'description': 'SYSID_MYGCS Allows restricting radio overrides to only come from my ground station',
  'range': ['1', '255'],
  'increment': None,
  'value': []},
 'PILOT_THR_FILT': {'name': 'PILOT_THR_FILT',
  'description': 'PILOT_THR_FILT Throttle filter cutoff (Hz) - active whenever altitude control is inactive - 0 to disable',
  'range': ['0', '10'],
  'increment': 0.5,
  'value': []},
 'PILOT_TKOFF_ALT': {'name': 'PILOT_TKOFF_ALT',
  'description': 'PILOT_TKOFF_ALT Altitude that altitude control modes will climb to when a takeoff is triggered with the throttle stick.',
  'range': ['0.0', '1000.0'],
  'increment': 10.0,
  'value': []},
 'PILOT_THR_BHV': {'name': 'PILOT_THR_BHV',
  'descript

In [3]:
configs['RTL_ALT']

{'name': 'RTL_ALT',
 'description': 'RTL_ALT The minimum alt above home the vehicle will climb to before returning.  If the vehicle is flying higher than this value it will return at its current altitude.',
 'range': ['200', '300000'],
 'increment': 1.0,
 'value': []}

In [4]:
# policies = None
with open(env_vars.PROJ_HOME + 'src/VSM/policy_descriptions') as csv_file:
    policies = {}
    file_data = csv.reader(csv_file, delimiter=';')
    headers = next(file_data)
    for row in file_data:
        row_dict = dict(zip(headers, row))

        configs[row[0]] = {
            'name': row_dict['Policy'],
            'description':row_dict['Description']
        } # put policies tgt with the configs
        # policies[row_dict['Policy']] = row_dict['Description']
# policies

In [5]:
configs['A.RTL2']

{'name': 'A.RTL2',
 'description': 'If the current altitude is greater or equal to RTL_ALT, current flight mode is RTL, and the current vehicle is not at home position, then the vehicle must move to the home position while maintaining the current altitude.'}

Construct Incidence Dictionary

In [6]:
configInciDict = {configName:0 for configName in configs}
configInciDict

{'SYSID_THISMAV': 0,
 'SYSID_MYGCS': 0,
 'PILOT_THR_FILT': 0,
 'PILOT_TKOFF_ALT': 0,
 'PILOT_THR_BHV': 0,
 'TELEM_DELAY': 0,
 'GCS_PID_MASK': 0,
 'RTL_ALT': 0,
 'RTL_CONE_SLOPE': 0,
 'RTL_SPEED': 0,
 'RTL_ALT_FINAL': 0,
 'RTL_CLIMB_MIN': 0,
 'RTL_LOIT_TIME': 0,
 'RTL_ALT_TYPE': 0,
 'FS_GCS_ENABLE': 0,
 'GPS_HDOP_GOOD': 0,
 'SUPER_SIMPLE': 0,
 'WP_YAW_BEHAVIOR': 0,
 'LAND_SPEED': 0,
 'LAND_SPEED_HIGH': 0,
 'PILOT_SPEED_UP': 0,
 'PILOT_ACCEL_Z': 0,
 'FS_THR_ENABLE': 0,
 'FS_THR_VALUE': 0,
 'THR_DZ': 0,
 'FLTMODE1': 0,
 'FLTMODE2': 0,
 'FLTMODE3': 0,
 'FLTMODE4': 0,
 'FLTMODE5': 0,
 'FLTMODE6': 0,
 'FLTMODE_CH': 0,
 'INITIAL_MODE': 0,
 'SIMPLE': 0,
 'LOG_BITMASK': 0,
 'ESC_CALIBRATION': 0,
 'TUNE': 0,
 'FRAME_TYPE': 0,
 'DISARM_DELAY': 0,
 'ANGLE_MAX': 0,
 'PHLD_BRAKE_RATE': 0,
 'PHLD_BRAKE_ANGLE': 0,
 'LAND_REPOSITION': 0,
 'FS_EKF_ACTION': 0,
 'FS_EKF_THRESH': 0,
 'FS_CRASH_CHECK': 0,
 'RC_SPEED': 0,
 'ACRO_BAL_ROLL': 0,
 'ACRO_BAL_PITCH': 0,
 'ACRO_TRAINER': 0,
 'THROW_MOT_START': 0,
 

In [7]:
allConfigDescTermsInciDict = {}

for configName in configInciDict:
    # if configName == "A.RTL2":
    print('Processing config', configName)
    line = configs[configName]['description']
    for char in ['~','#','?','\\n','`','*','!', '}', '=', '|', '@', '+', '-', '(', ')', '<', '>', '/', ':', '"', '"', '\'', '.', ',']:
        line = line.replace(char, "  ")
    terms = line.split()

    filtered = [t.lower() for t in terms if t.lower() not in stop_words]

    for term in filtered:
        if not term.isnumeric() and term != '':
            if term not in allConfigDescTermsInciDict:
                allConfigDescTermsInciDict[term] = copy.deepcopy(configInciDict)

            allConfigDescTermsInciDict[term][configName] += 1

Processing config SYSID_THISMAV
Processing config SYSID_MYGCS
Processing config PILOT_THR_FILT
Processing config PILOT_TKOFF_ALT
Processing config PILOT_THR_BHV
Processing config TELEM_DELAY
Processing config GCS_PID_MASK
Processing config RTL_ALT
Processing config RTL_CONE_SLOPE
Processing config RTL_SPEED
Processing config RTL_ALT_FINAL
Processing config RTL_CLIMB_MIN
Processing config RTL_LOIT_TIME
Processing config RTL_ALT_TYPE
Processing config FS_GCS_ENABLE
Processing config GPS_HDOP_GOOD
Processing config SUPER_SIMPLE
Processing config WP_YAW_BEHAVIOR
Processing config LAND_SPEED
Processing config LAND_SPEED_HIGH
Processing config PILOT_SPEED_UP
Processing config PILOT_ACCEL_Z
Processing config FS_THR_ENABLE
Processing config FS_THR_VALUE
Processing config THR_DZ
Processing config FLTMODE1
Processing config FLTMODE2
Processing config FLTMODE3
Processing config FLTMODE4
Processing config FLTMODE5
Processing config FLTMODE6
Processing config FLTMODE_CH
Processing config INITIAL_MO

In [9]:
allConfigDescTermsInciDf = pd.DataFrame(allConfigDescTermsInciDict)
allConfigDescTermsInciDf.to_csv(env_vars.PROJ_HOME + 'src/VSM/allConfigTermsInciDict.csv')

<h3>TF-IDF

Load Incidence Dictionary

In [10]:
loadedAllConfigDescTermsInciDf = pd.read_csv(env_vars.PROJ_HOME + 'src/VSM/allConfigTermsInciDict.csv', index_col=0)
allConfigDescTermsInciDict = loadedAllConfigDescTermsInciDf.to_dict()
loadedAllConfigDescTermsInciDf

,sysid_thismav,allows,setting,individual,mavlink,system,id,vehicle,distinguish,others,...,k,computed,middle,think,counterclockwise,pilot’s,overridden,way,indicated,vehicle’s
SYSID_THISMAV,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
SYSID_MYGCS,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PILOT_THR_FILT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PILOT_TKOFF_ALT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PILOT_THR_BHV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A.RC.FS1,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
A.RC.FS2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A.CHUTE1,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,1
A.GPS.FS1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


Test the occurence of the term 'altitude' at a document  

In [12]:
def termFrequency(allTermsInciDict, doc):
    tfDict = {}

    for term in allTermsInciDict:
        tfDict[term] = allTermsInciDict[term][doc]
    
    return tfDict

Initiate blank dictionary of keep count of documents

In [13]:
configInciDict = {h:0 for h in loadedAllConfigDescTermsInciDf.index.values.tolist()}
configInciDict

{'SYSID_THISMAV': 0,
 'SYSID_MYGCS': 0,
 'PILOT_THR_FILT': 0,
 'PILOT_TKOFF_ALT': 0,
 'PILOT_THR_BHV': 0,
 'TELEM_DELAY': 0,
 'GCS_PID_MASK': 0,
 'RTL_ALT': 0,
 'RTL_CONE_SLOPE': 0,
 'RTL_SPEED': 0,
 'RTL_ALT_FINAL': 0,
 'RTL_CLIMB_MIN': 0,
 'RTL_LOIT_TIME': 0,
 'RTL_ALT_TYPE': 0,
 'FS_GCS_ENABLE': 0,
 'GPS_HDOP_GOOD': 0,
 'SUPER_SIMPLE': 0,
 'WP_YAW_BEHAVIOR': 0,
 'LAND_SPEED': 0,
 'LAND_SPEED_HIGH': 0,
 'PILOT_SPEED_UP': 0,
 'PILOT_ACCEL_Z': 0,
 'FS_THR_ENABLE': 0,
 'FS_THR_VALUE': 0,
 'THR_DZ': 0,
 'FLTMODE1': 0,
 'FLTMODE2': 0,
 'FLTMODE3': 0,
 'FLTMODE4': 0,
 'FLTMODE5': 0,
 'FLTMODE6': 0,
 'FLTMODE_CH': 0,
 'INITIAL_MODE': 0,
 'SIMPLE': 0,
 'LOG_BITMASK': 0,
 'ESC_CALIBRATION': 0,
 'TUNE': 0,
 'FRAME_TYPE': 0,
 'DISARM_DELAY': 0,
 'ANGLE_MAX': 0,
 'PHLD_BRAKE_RATE': 0,
 'PHLD_BRAKE_ANGLE': 0,
 'LAND_REPOSITION': 0,
 'FS_EKF_ACTION': 0,
 'FS_EKF_THRESH': 0,
 'FS_CRASH_CHECK': 0,
 'RC_SPEED': 0,
 'ACRO_BAL_ROLL': 0,
 'ACRO_BAL_PITCH': 0,
 'ACRO_TRAINER': 0,
 'THROW_MOT_START': 0,
 

Create Inverse Document Frequency for all the terms in the Config and the Policies

In [14]:
nConfig = len(configInciDict)
configIDF={k:(nConfig/(sum([1 for e in v.values() if e != 0]))) for k,v in  allConfigDescTermsInciDict.items()} 
configIdfDf = pd.DataFrame(configIDF, index=['IDF'])
configIdfDf.to_csv(env_vars.PROJ_HOME + 'src/VSM/configIDF.csv')
configIdfDf

,sysid_thismav,allows,setting,individual,mavlink,system,id,vehicle,distinguish,others,...,k,computed,middle,think,counterclockwise,pilot’s,overridden,way,indicated,vehicle’s
IDF,3115.0,75.97561,33.494624,3115.0,59.903846,100.483871,124.6,16.747312,1557.5,3115.0,...,1557.5,3115.0,3115.0,778.75,1038.333333,3115.0,3115.0,3115.0,3115.0,3115.0


<h3>Vector Space Modelling

In [15]:
try:
    os.remove(env_vars.PROJ_HOME + 'src/VSM/ConfigVSM.csv')
except OSError:
    pass

zeroesConfig = [0.0]*len(configIDF)

configIdfTemp={'config_idf':configIDF.values()}
for d in configInciDict:
    configIdfTemp[d] = zeroesConfig

configVsmDf = pd.DataFrame(configIdfTemp, index=configIDF.keys())

i = 0
for config in configInciDict:
    term_frequencies = termFrequency(allConfigDescTermsInciDict, config)
    print(i, '/', len(configInciDict), ':', config)
    tf_idf = {}
    tf_idf['config'] = [config]

    for term, tf in term_frequencies.items():
        # configVsmDf[config][term] = configVsmDf['config_idf'][term] * float(tf)
        tf_idf[term] = [float(tf) * configIDF[term]]
    if i == 0:
        pd.DataFrame(tf_idf).to_csv(env_vars.PROJ_HOME + 'src/VSM/ConfigVSM.csv', mode='a', index=False)
    else:
        pd.DataFrame(tf_idf).to_csv(env_vars.PROJ_HOME + 'src/VSM/ConfigVSM.csv', mode='a', index=False, header=False)

    i += 1

0 / 3115 : SYSID_THISMAV
1 / 3115 : SYSID_MYGCS
2 / 3115 : PILOT_THR_FILT
3 / 3115 : PILOT_TKOFF_ALT
4 / 3115 : PILOT_THR_BHV
5 / 3115 : TELEM_DELAY
6 / 3115 : GCS_PID_MASK
7 / 3115 : RTL_ALT
8 / 3115 : RTL_CONE_SLOPE
9 / 3115 : RTL_SPEED
10 / 3115 : RTL_ALT_FINAL
11 / 3115 : RTL_CLIMB_MIN
12 / 3115 : RTL_LOIT_TIME
13 / 3115 : RTL_ALT_TYPE
14 / 3115 : FS_GCS_ENABLE
15 / 3115 : GPS_HDOP_GOOD
16 / 3115 : SUPER_SIMPLE
17 / 3115 : WP_YAW_BEHAVIOR
18 / 3115 : LAND_SPEED
19 / 3115 : LAND_SPEED_HIGH
20 / 3115 : PILOT_SPEED_UP
21 / 3115 : PILOT_ACCEL_Z
22 / 3115 : FS_THR_ENABLE
23 / 3115 : FS_THR_VALUE
24 / 3115 : THR_DZ
25 / 3115 : FLTMODE1
26 / 3115 : FLTMODE2
27 / 3115 : FLTMODE3
28 / 3115 : FLTMODE4
29 / 3115 : FLTMODE5
30 / 3115 : FLTMODE6
31 / 3115 : FLTMODE_CH
32 / 3115 : INITIAL_MODE
33 / 3115 : SIMPLE
34 / 3115 : LOG_BITMASK
35 / 3115 : ESC_CALIBRATION
36 / 3115 : TUNE
37 / 3115 : FRAME_TYPE
38 / 3115 : DISARM_DELAY
39 / 3115 : ANGLE_MAX
40 / 3115 : PHLD_BRAKE_RATE
41 / 3115 : PHLD_BR

<h3>Cosine Similarity

Load Vector Space Model

In [17]:
loadedConfigVsm = pd.read_csv(env_vars.PROJ_HOME + 'src/VSM/ConfigVSM.csv', index_col=0)
VsmT = loadedConfigVsm.T
VsmT

config,SYSID_THISMAV,SYSID_MYGCS,PILOT_THR_FILT,PILOT_TKOFF_ALT,PILOT_THR_BHV,TELEM_DELAY,GCS_PID_MASK,RTL_ALT,RTL_CONE_SLOPE,RTL_SPEED,...,A.BRAKE1,A.DRIFT1,A.LOITER1,A.GUIDED1,A.SPORT1,A.RC.FS1,A.RC.FS2,A.CHUTE1,A.GPS.FS1,A.GPS.FS2
sysid_thismav,3115.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
allows,75.975610,75.97561,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
setting,33.494624,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
individual,3115.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mavlink,59.903846,0.00000,0.0,0.0,0.0,0.0,59.903846,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pilot’s,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
overridden,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
way,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,3115.0,0.0,0.0,0.0,0.0,0.0,0.0
indicated,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3115.0,0.0,0.0,0.0,0.0,0.0


Test out a column vector inside the DataFrame

In [18]:
VsmT['A.RTL2']

sysid_thismav    0.0
allows           0.0
setting          0.0
individual       0.0
mavlink          0.0
                ... 
pilot’s          0.0
overridden       0.0
way              0.0
indicated        0.0
vehicle’s        0.0
Name: A.RTL2, Length: 5557, dtype: float64

Cosine Similarity Function

In [19]:
def cos_sim(v1,v2):
    return 1 - spatial.distance.cosine(v1,v2)

Make a list of Policies

In [20]:
policies = []
with open(env_vars.PROJ_HOME + 'src/VSM/policy_descriptions') as csv_file:
    file_data = csv.reader(csv_file, delimiter=';')
    headers = next(file_data)
    for row in file_data:
        policies.append(row[0])
policies

['A.RTL1',
 'A.RTL2',
 'A.RTL3',
 'A.RTL4',
 'A.FLIP1',
 'A.FLIP2',
 'A.FLIP3',
 'A.FLIPGeneral',
 'A.ALT_HOLD1',
 'A.ALT_HOLD2',
 'A.CIRCLE1',
 'A.CIRCLE2',
 'A.CIRCLE3',
 'A.CIRCLE4',
 'A.CIRCLE5',
 'A.CIRCLE6',
 'A.CIRCLE7',
 'A.LAND1',
 'A.LAND2',
 'A.AUTO1',
 'A.BRAKE1',
 'A.DRIFT1',
 'A.LOITER1',
 'A.GUIDED1',
 'A.SPORT1',
 'A.RC.FS1',
 'A.RC.FS2',
 'A.CHUTE1',
 'A.GPS.FS1',
 'A.GPS.FS2']

Calculate Cosine Similarity with all the Configs

In [21]:
rankings = {}
for policy in policies:
    if policy not in rankings:
        rankings[policy] = []
    for column in VsmT:
        if column not in policies:
            col = VsmT[column].values
            rankings[policy].append((column, cos_sim(VsmT[column].values, VsmT[policy].values)))
    rankings[policy] = sorted(rankings[policy], key=lambda x: x[1], reverse=True)

rankingsDf = pd.DataFrame(rankings)
rankingsDf.to_csv(env_vars.PROJ_HOME + 'src/VSM/rankings_config.csv')
rankingsDf

,A.RTL1,A.RTL2,A.RTL3,A.RTL4,A.FLIP1,A.FLIP2,A.FLIP3,A.FLIPGeneral,A.ALT_HOLD1,A.ALT_HOLD2,...,A.BRAKE1,A.DRIFT1,A.LOITER1,A.GUIDED1,A.SPORT1,A.RC.FS1,A.RC.FS2,A.CHUTE1,A.GPS.FS1,A.GPS.FS2
0,"(RTL_ALT, 0.4576311070278485)","(RTL_ALT, 0.1607295101641426)","(CHUTE_ALT_MIN, 0.004454105287990839)","(DISARM_DELAY, 0.14246503594694027)","(H_RSC_THRCRV_0, 0.07225644329900738)","(H_RSC_THRCRV_0, 0.17732094112658803)","(CAN_SLCAN_TIMOUT, 0.051874433381600915)","(SID_T_REC, 0.14416908505375692)","(FOLL_OFS_TYPE, 0.028060832329158014)","(WPNAV_SPEED, 0.018785908457365608)",...,"(LOIT_BRK_DELAY, 0.30876066246094247)","(FS_EKF_ACTION, 0.3570689061728056)","(WPNAV_SPEED, 0.1253110003004293)","(AVOID_MARGIN, 0.15995092248899223)","(PILOT_SPEED_UP, 0.09676846281258755)","(FS_THR_VALUE, 0.5288596952896151)","(FS_THR_VALUE, 0.8337891307289818)","(CHUTE_ALT_MIN, 0.2364395249260144)","(CAM_FEEDBACK_POL, 0.06704109936286717)","(RALLY_INCL_HOME, 0.07356649544335936)"
1,"(EK2_WIND_PSCALE, 0.06369659801159155)","(RTL_ALT_TYPE, 0.01111692782157625)","(RTL_ALT, 0.004309664107046918)","(OSD_DSARM_SCR, 0.07089743643765556)","(H_RSC_THRCRV_100, 0.07225644329900738)","(H_RSC_THRCRV_100, 0.17732094112658803)","(H_COL_LAND_MIN, 0.008989584010298812)","(CAN_SLCAN_TIMOUT, 0.11670169828805865)","(INS_HNTC2_ATT, 0.01824284392604869)","(WPNAV_SPEED_DN, 0.017783094617215722)",...,"(WPNAV_TER_MARGIN, 0.015683362530046985)","(FS_DR_ENABLE, 0.13411456717902503)","(WPNAV_SPEED_DN, 0.11862175204238046)","(THROW_NEXTMODE, 0.06804930774842477)","(PILOT_SPEED_DN, 0.09244582413621505)","(FS_DR_ENABLE, 0.21168469773072351)","(RALLY_INCL_HOME, 0.045088184555095)","(WPNAV_SPEED_UP, 0.149852047709788)","(INS_HNTC2_OPTS, 0.06575784065159929)","(BRD_HEAT_LOWMGN, 0.05537187034444013)"
2,"(EK3_WIND_PSCALE, 0.06369659801159155)","(ARMING_ACCTHRESH, 0.007808555096871506)","(OA_DB_ALT_MIN, 0.002898802058224015)","(FS_CRASH_CHECK, 0.06840498194068367)","(TMODE_FLAGS, 0.06862664535182217)","(H_RSC_THRCRV_25, 0.15774288568625883)","(RALLY_INCL_HOME, 0.006191198488249006)","(H_RSC_CLDWN_TIME, 0.04655947743937605)","(INS_HNTCH_ATT, 0.01824284392604869)","(H_COL_LAND_MIN, 0.014522032125654283)",...,"(LOIT_BRK_ACCEL, 0.015166313606941206)","(FFT_HMNC_FIT, 0.10194190455234642)","(ATC_ANG_LIM_TC, 0.11578094620464818)","(COMPASS_USE, 0.01003912994178946)","(FRSKY_DNLINK_ID, 0.017985598347931897)","(DISARM_DELAY, 0.048313185042945905)","(BRD_HEAT_LOWMGN, 0.03393687702811832)","(LGR_DEPLOY_ALT, 0.11161039848702203)","(INS_HNTCH_OPTS, 0.06575784065159929)","(COMPASS_USE2, 0.04843963276548091)"
3,"(H_COLYAW, 0.05354084734700271)","(RTL_OPTIONS, 0.007700584966067847)","(ADSB_LIST_ALT, 0.00241435049213079)","(ARMING_ACCTHRESH, 0.03967626355760967)","(H_RSC_THRCRV_25, 0.06427858888519011)","(H_RSC_THRCRV_50, 0.15774288568625883)","(OA_OPTIONS, 0.005128801539668548)","(COMPASS_ORIENT, 0.029596563363595618)","(FOLL_ALT_TYPE, 0.010366948414156418)","(ATC_ANG_LIM_TC, 0.01237601887431583)",...,"(AVOID_BEHAVE, 0.012050678943189785)","(CAM_RC_CONTROL, 0.05049916887350048)","(FENCE_MARGIN, 0.07768471724884396)","(COMPASS_USE2, 0.008696465425387379)","(FRSKY_DNLINK1_ID, 0.01716468520293024)","(OSD_DSARM_SCR, 0.024042958631327993)","(BRD_SAFETYENABLE, 0.028517016343282964)","(DID_BARO_ACC, 0.03513725736286688)","(CAM_TRIGG_TYPE, 0.05836492960007755)","(BRD_SAFETYENABLE, 0.04652875190201611)"
4,"(ARMING_ACCTHRESH, 0.011518276827259144)","(WP_YAW_BEHAVIOR, 0.007616394087812561)","(EK2_WIND_PSCALE, 0.0021059111711883016)","(SERVO_BLH_AUTO, 0.033721623101258924)","(H_RSC_THRCRV_50, 0.06427858888519011)","(H_RSC_THRCRV_75, 0.08602360672750742)","(CAM_DURATION, 0.0045965145505600935)","(COMPASS_ORIENT2, 0.02622956918501751)","(FOLL_YAW_BEHAVE, 0.009718047216296544)","(FENCE_MARGIN, 0.01164604849753248)",...,"(SCR_DIR_DISABLE, 0.011697899554373503)","(ESC_CALIBRATION, 0.04318629108160155)","(AHRS_COMP_BETA, 0.07625940480722992)","(COMPASS_USE3, 0.008145069691250395)","(FRSKY_DNLINK2_ID, 0.017157750413581363)","(FS_CR